In [1]:
!wget http://www.manythings.org/anki/pes-eng.zip
!unzip -q pes-eng.zip

--2023-12-01 19:26:21--  http://www.manythings.org/anki/pes-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 150371 (147K) [application/zip]
Saving to: ‘pes-eng.zip.3’

pes-eng.zip.3       100%[===================>] 146.85K  44.2KB/s    in 3.3s    

2023-12-01 19:26:25 (44.2 KB/s) - ‘pes-eng.zip.3’ saved [150371/150371]

replace _about.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [2]:
text_file="pes.txt"
with open(text_file) as f:
    lines=f.read().split("\n")[:-1]
text_pairs=[]

In [3]:
for line in lines :
    english, persian ,null = line.split("\t")
    persian = "[start] " + persian + " [end]"
    text_pairs.append((english,persian))

In [4]:
import random
print(random.choice(text_pairs))

('Marriage is the main cause of all divorces.', '[start] ازدواج علت اصلی همهٔ طلاقهاست. [end]')


In [5]:
import random
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

In [6]:
import tensorflow as tf
import string
import re
from tensorflow import keras
from tensorflow.keras import layers

strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")

vocab_size = 2500
sequence_length = 15

source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_english_texts = [pair[0] for pair in train_pairs]
train_persian_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_persian_texts)

2023-12-01 19:27:37.748803: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-01 19:27:38.643723: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [7]:
batch_size = 64

def format_dataset(eng, pes):
    eng = source_vectorization(eng)
    pes = target_vectorization(pes)
    return ({
        "english": eng,
        "persian": pes[:, :-1],
    }, pes[:, 1:])

def make_dataset(pairs):
    eng_texts, pes_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    pes_texts = list(pes_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, pes_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [8]:
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['persian'].shape: {inputs['persian'].shape}")
    print(f"targets.shape: {targets.shape}")

inputs['english'].shape: (64, 15)
inputs['persian'].shape: (64, 15)
targets.shape: (64, 15)


2023-12-01 19:27:38.936462: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype resource
	 [[{{node Placeholder/_10}}]]
2023-12-01 19:27:38.936725: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_12' with dtype string
	 [[{{node Placeholder/_12}}]]
2023-12-01 19:27:38.966442: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.ca

In [9]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        else:
            padding_mask = mask
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

<div dir="rtl">
Positional embeddings (تعبیه‌های موقعیتی) یکی از عناصر کلیدی در مدل‌های پردازش زبان طبیعی (NLP) مبتنی بر ترتیب مانند Transformer است. این تعبیه‌ها برای افزودن اطلاعات مرتبط با موقعیت (ترتیب) کلمات در جمله به بردارهای ویژگی هر کلمه استفاده می‌شوند.

در معماری Transformer، که به طور خاص برای پردازش دنباله‌های طولانی مانند جملات زبان طبیعی ایجاد شده است، از self-attention mechanism استفاده می‌شود. این مکانیزم به شبکه این امکان را می‌دهد تا به ویژگی‌های هر کلمه با توجه به تمام کلمات دنباله وزن دهد. با این حال، اطلاعات مربوط به ترتیب کلمات در جمله از دست می‌روند چرا که مدل به تنهایی اطلاعات ترتیبی را نمی‌فهمد.

برای حل این مشکل، تعبیه‌های موقعیتی اضافه می‌شوند. این تعبیه‌ها به هر کلمه یا موقعیت در جمله یک بردار مخصوص اختصاص می‌دهند. این بردارها به عنوان ورودی به لایه‌های Transformer اضافه می‌شوند و با ویژگی‌های متناظر هر کلمه جمع می‌شوند. به این ترتیب، مدل قادر به درک بهتر ترتیب موقعیتی کلمات می‌شود.

تعبیه‌های موقعیتی معمولاً با استفاده از توابع ریاضی یا جدول‌های ثابت ایجاد می‌شوند و در مرحله اولیه آموزش مدل به آنها یادگیری نمی‌شود. این اطلاعات موقعیتی بهترین شکل را برای ورودی به شبکه فراهم می‌کنند و به مدل کمک می‌کنند که وابستگی‌های زمانی را در داده‌های ورودی بشناسد.
</div>

In [10]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

In [11]:
from tensorflow.keras.layers import LayerNormalization, Layer, Dense, ReLU, Dropout

# Implementing the Add & Norm Layer
class AddNormalization(Layer):
    def __init__(self, **kwargs):
        super(AddNormalization, self).__init__(**kwargs)
        self.layer_norm = LayerNormalization()  # Layer normalization layer

    def call(self, x, sublayer_x):
        # The sublayer input and output need to be of the same shape to be summed
        add = x + sublayer_x

        # Apply layer normalization to the sum
        return self.layer_norm(add)


In [12]:
class FeedForward(tf.keras.layers.Layer):
  def __init__(self, d_model, dff, dropout_rate=0.1):
    super().__init__()
    self.seq = tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),
      tf.keras.layers.Dense(d_model),
      tf.keras.layers.Dropout(dropout_rate)
    ])
    self.add = tf.keras.layers.Add()
    self.layer_norm = tf.keras.layers.LayerNormalization()

  def call(self, x):
    x = self.add([x, self.seq(x)])
    x = self.layer_norm(x)
    return x

<div dir="rtl">

تفاوت attention و self attention

Attention و self-attention (یا به نام دیگر scaled dot-product attention) دو ویژگی مرتبط با مدل‌های Transformer هستند، اما مفاهیم متفاوتی دارند.

1. **Attention:**
   در Attention، ورودی شبکه به تعداد سرهایی (heads) تقسیم می‌شود و سپس هر سر به صورت جداگانه با ویژگی‌های ورودی ترکیب می‌شود. به عبارت دیگر، هر سر به دنبال توجه به نقاط مختلف دنباله است و در نهایت خروجی‌های این سرها با یکدیگر ترکیب می‌شوند.

2. **Self-Attention (Scaled Dot-Product Attention):**
   در self-attention، توجه به خود (self) انجام می‌شود، به این معنی که هر عنصر از دنباله به سایر عناصر توجه می‌کند. برای هر ورودی، توجه به تمام ورودی‌های دیگر با استفاده از محاسبه dot-product انجام می‌شود. این نوع توجه به شبکه این امکان را می‌دهد که اطلاعات متقابل بین همه عناصر دنباله را دریافت کند.

   در self-attention، برای کنترل مقیاس و تعادل، dot-product نرمال شده می‌شود و سپس به عنوان وزن‌های توجه برای ترکیب خطی ویژگی‌ها استفاده می‌شود.

در واقع، self-attention یک نوع از attention است که در آن توجه به خود (self) و همچنین توجه به نقاط مختلف دنباله انجام می‌شود. این امکان باعث می‌شود که شبکه قادر به درک تعاملات داخلی در دنباله باشد و ارتباطات موقعیتی و ترتیبی بین عناصر را در نظر گیرد.

در پایین ما از دو مدل برای مقایسه هر دو روش با استفاده از ۸ سر استفاده میکنیم .

برای توضیح دقیق تر یک ویس در تلگرام ارسال خواهد شد.


In [13]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

<div dir="rtl">
در این بخش با استفاده از توضیحات بالا ما از ۸ سر برای attention استفاده میکنیم

In [18]:
embed_dim = 256
dense_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="persian")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [15]:
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
transformer.fit(train_ds, epochs=30, validation_data=val_ds)

Epoch 1/30


2023-12-01 19:27:39.349049: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_13' with dtype int64
	 [[{{node Placeholder/_13}}]]
2023-12-01 19:27:39.349299: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_13' with dtype int64
	 [[{{node Placeholder/_13}}]]


35/35 [==============================] - ETA: 0s - loss: 5.3251 - accuracy: 0.2526

2023-12-01 19:27:47.325630: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_6' with dtype resource
	 [[{{node Placeholder/_6}}]]
2023-12-01 19:27:47.331524: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_12' with dtype string
	 [[{{node Placeholder/_12}}]]


35/35 [==============================] - 9s 212ms/step - loss: 5.3251 - accuracy: 0.2526 - val_loss: 4.1337 - val_accuracy: 0.3600
Epoch 2/30
35/35 [==============================] - 6s 173ms/step - loss: 4.5914 - accuracy: 0.3146 - val_loss: 3.8885 - val_accuracy: 0.3987
Epoch 3/30
35/35 [==============================] - 6s 178ms/step - loss: 4.1891 - accuracy: 0.3533 - val_loss: 3.7678 - val_accuracy: 0.3878
Epoch 4/30
35/35 [==============================] - 7s 186ms/step - loss: 3.8868 - accuracy: 0.3841 - val_loss: 3.7344 - val_accuracy: 0.3769
Epoch 5/30
35/35 [==============================] - 6s 180ms/step - loss: 3.5503 - accuracy: 0.4213 - val_loss: 3.5739 - val_accuracy: 0.4026
Epoch 6/30
35/35 [==============================] - 6s 176ms/step - loss: 3.2690 - accuracy: 0.4556 - val_loss: 3.5283 - val_accuracy: 0.4212
Epoch 7/30
35/35 [==============================] - 6s 179ms/step - loss: 3.0145 - accuracy: 0.4869 - val_loss: 3.7216 - val_accuracy: 0.4066
Epoch 8/30
35/35 

<div dir="rtl">

در یادگیری بالا ما به دقت 40 درصد رسیدیم،از آنجایی که تعداد جملات ما بسیار کم بود این دقت تا حد زیادی قابل قبول از و اور فیت در آن دیده نمیشود.

In [16]:
import numpy as np
pes_vocab = target_vectorization.get_vocabulary()
pes_index_lookup = dict(zip(range(len(pes_vocab)), pes_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization(
            [decoded_sentence])[:, :-1]
        predictions = transformer(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = pes_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence


In [17]:
    print(decode_sequence("we are going out"))

[start] ما چه می روی؟ [end]
